# Pipeline for Recovering Neural Activity from Calcium Imaging Data

### Relevant Citations / Dependencies
**Motion Correction ->** Suite2P Package from X <br>
**ROI Identifcation ->** Cellpose Package from X <br>
**Signal Extraction ->** FISSA Package from X <br>
**Source-Separation ->** FISSA Package from X <br>
**Spike Inference ->** Cascade Package from X <br>

### Import Modules

In [ ]:
# FISSA: Signal Extraction & Source-Separation
from AnalysisModules.FissaAnalysis import FissaModule
# Assorted Static Processing Functions
from AnalysisModules.StaticProcessing import smoothTraces_TiffOrg, calculate_dFoF, mergeTraces, detrendTraces

### Instantiation

In [ ]:
# Instantiate Fissa Module & Sub-Modules (preparation, experiment (separation), and ProcessedTraces)
Data = FissaModule(data_folder="D:\\M4618_Second_Round\\Encoding", index_file = "D:\\M4618_Second_Round\\Encoding\\suite2p\\plane0\\Debug_Neuronal_Index.csv")
# Initialize
Data.pruneNonNeuronalROIs()
Data.initializeFissa()


### Extract Traces

In [ ]:
Data.extractTraces()

### Save Progress

In [ ]:
Data.saveFissaPrep()

### Pre-Process Traces

In [ ]:
Data.ProcessedTraces.smoothed_raw = smoothTraces_TiffOrg(Data.preparation.raw, niter=50, kappa=150, gamma=0.15)[0]

### Save Progress

In [ ]:
Data.saveProcessedTraces()

### Use Pre-Processed Traces for Source-Separation

In [ ]:
Data.preparation.raw = Data.ProcessedTraces.smoothed_raw.copy()
Data.passPrepToFissa()

### Source-Separate Traces

In [ ]:
Data.separateTraces()

### Save Progress

In [ ]:
Data.saveFissaSep()

### Post-Process Traces

In [ ]:
# Calculate Fo/F
Data.ProcessedTraces.dFoF_result = calculate_dFoF(Data.experiment.result, Data.frame_rate, raw=Data.preparation.raw, merge_after=False)
# Condense the ROI Traces for each Trial into a Single Matrix
Data.ProcessedTraces.merged_dFoF_result = mergeTraces(Data.ProcessedTraces.dFoF_result)
# Detrend the Traces
Data.ProcessedTraces.detrended_merged_dFoF_result = detrendTraces(Data.ProcessedTraces.merged_dFoF_result, order=4, plot=False)

### Save Progress

In [ ]:
Data.saveProcessedTraces()

## Infer Firing Rates & Approximate Spikes

### Import Modules

In [ ]:
# CASCADE: Spike Inference
from AnalysisModules.CascadeAnalysis import CascadeModule
# Assorted Static Processing Functions
from AnalysisModules.StaticProcessing import calculateFiringRate
# Assorted Static Utility Functions
from AnalysisModules.StaticUtilities import pullModels

### Instantiation

In [ ]:
# Pass the Post-Processed Traces
Data.Cascade = CascadeModule(Data.ProcessedTraces.detrended_merged_dFoF_result, Data.frame_rate, model_folder="C:\\ProgramData\\Anaconda3\\envs\\suite2p\\Pretrained_models")

### Model Selection

In [ ]:
# Pull Available Models
list_of_models = pullModels(Data.Cascade.model_folder)
# Select Model: If you know what model you want, you should use the string instead.
# Download Model: You will need to downlaod the model before use
# from AnalysisModules.StaticUtilities import downloadModel
# This model is GLobal_EXC_10Hz_smoothing_100ms
Data.Cascade.model_name = list_of_models[21]

### Infer Firing Rates

In [ ]:
# Infer Spike Probability
Data.Cascade.predictSpikeProb()
# Calculate Firing Rates
Data.Cascade.ProcessedInferences.firing_rates = calculateFiringRate(Data.Cascade.spike_prob, Data.Cascade.frame_rate)

### Save Progress

In [ ]:
Data.Cascade.saveSpikeProb(Data.output_folder)
Data.Cascade.saveProcessedInferences(Data.output_folder)

### Infer Discrete Spike Times

In [ ]:
# If you have an infinite loop:
# you are looking at a neuron that should have been dropped for
# (1) exceedingly poor SNR
# (2) major trend in the data (e.g., lots of bleaching)
# (3) abnormal phenotype for particular model-- think of training your own!
Data.Cascade.inferDiscreteSpikes()

### Save Progress

In [ ]:
Data.Cascade.saveSpikeInference(Data.output_folder)

### Export to MATLab for MATLab Analyses

In [ ]:
Data.Cascade.exportSpikeProb(Data.output_folder)
Data.Cascade.exportSpikeInference(Data.output_folder)

### Visualize Noise Histogram

In [ ]:
from AnalysisModules.StaticPlotting import plotNoise
plotNoise(Data.Cascade.traces, Data.Cascade.frame_rate)

### Visualize Spike Inference

In [ ]:
from AnalysisModules.StaticPlotting import plotSpikeInference
plotSpikeInference(Data.Cascade.spike_prob, Data.Cascade.spike_time_estimates, Data.Cascade.traces, Data.Cascade.frame_rate)